# 特徴量エンジニアリング概要

## 本プロジェクトの目的

機械学習のパフォーマンス改善に向けて重要な特徴量エンジニアリングの手法を，実装例を交えて紹介する．  
主に特徴量エンジニアリングについて述べるが，一部特徴量選択についても触れる．

## 特徴量エンジニアリングとは

機械学習における特徴量とは，分析対象を測定することが可能な変数を指す．データセットでは特徴量は列として表記されることが多い．

データセットに含める特徴量の質が，機械学習モデルの精度に影響し，ひいては機械学習を活用する場合に得るインサイトの質に大きく影響する．

データセットの質を改善する為に，特徴量選択や特徴量エンジニアリングが実施される．  
特徴量選択は分析対象に関連する特徴量に重点を置き，無関係な特徴量を取り除くプロセスを指す．特徴量エンジニアリングは，既存の特徴量をもとに新たな特徴量を構築してデータセットに追加することを指す．

## 特徴量選択と特徴量エンジニアリングの具体例

特徴エンジニアリングの手法の詳細は他のNotebookに記載するが，本節では特徴量選択と特徴量エンジニアリングの違いを説明する為の具体例を示す．

データセットはFlood Modeling Datasetを使用し，論文[Time Series Extrinsic Regression](https://arxiv.org/abs/2006.12672)のSVR Optimisedの条件に対してtsfreshによる特徴量選択及び特徴量エンジニアリングを試行する．

学習パラメータは論文通り，下記パラメータに対して3-Folds Cross ValidationのGridSearchのベストモデルを採用する．

|Parameters|Values|
|:--|:--|
|Kernel|RBF, Sigmoid|
|gamma|0.001, 0.01, 0.1, 1|
|C|0.1, 1, 10, 100|


### 実装例

In [1]:
# --- ローカルモジュールの更新を自動で読み込む ---
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from lib.dataloader.flood_modeling import load_flood_modeling

#### データセットダウンロード

In [3]:
if (not os.path.exists("flood_modeling_datasets")):
    !mkdir -p "flood_modeling_datasets" ; \
        cd flood_modeling_datasets ; \
        wget "https://zenodo.org/record/3902694/files/FloodModeling1_TEST.ts" ; \
        wget "https://zenodo.org/record/3902694/files/FloodModeling1_TRAIN.ts" ; \
        ls
else:
    print('[INFO] Dataset flood_modeling_datasets is already exist')

[INFO] Dataset flood_modeling_datasets is already exist


In [4]:
train_ts = os.path.join('flood_modeling_datasets', 'FloodModeling1_TRAIN.ts')
test_ts = os.path.join('flood_modeling_datasets', 'FloodModeling1_TEST.ts')
x_train, y_train, x_test, y_test = load_flood_modeling(train_ts, test_ts)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(471, 266)
(471,)
(202, 266)
(202,)


#### 3-Flods Cross ValidationとGridSearchでモデルを学習する

In [5]:
def rmse(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return rmse

params = {
    'kernel': ['rbf', 'sigmoid'],
    'gamma': [0.001, 0.01, 0.1, 1],
    'C': [0.1, 1, 10, 100]
}
model_svr = GridSearchCV(
    svm.SVR(),
    params,
    cv=KFold(n_splits=3, shuffle=True, random_state=1234),
    scoring=make_scorer(rmse, greater_is_better=False))
model_svr.fit(x_train, y_train)

print('[INFO] Best params: {}'.format(model_svr.best_params_))
print('[INFO] Best score: {}'.format(-model_svr.best_score_))

[INFO] Best params: {'C': 0.1, 'gamma': 0.01, 'kernel': 'sigmoid'}
[INFO] Best score: 0.04293317276271991


#### テストデータで評価

In [6]:
prediction = model_svr.predict(x_test)
print(rmse(y_test, prediction))

0.046303583075482053


論文[Time Series Extrinsic Regression](https://arxiv.org/abs/2006.12672)の実験結果がRMSE=0.05なので，再現できた．

## Reference

* [特徴量変数](https://www.datarobot.com/jp/wiki/feature/)
* [特徴量の選択](https://www.datarobot.com/jp/wiki/feature-selection/)
* [特徴量エンジニアリング](https://www.datarobot.com/jp/wiki/feature-engineering/)
* [データインサイト](https://www.datarobot.com/jp/wiki/insights/)
* [Awesome Public Datasets](https://github.com/awesomedata/awesome-public-datasets)
* [Feature-Engineeringのリンク集めてみた](https://qiita.com/squash/items/667f8cda16c76448b0f4)
* [DataFrameで特徴量作るのめんどくさ過ぎる。。featuretoolsを使って自動生成したろ](https://qiita.com/Hyperion13fleet/items/4eaca365f28049fe11c7)
* [時系列データから自動で特徴抽出するライブラリ tsfresh](https://qiita.com/yuko1658/items/871df86f99a9134cc9ef)
* [特徴量選択のまとめ](https://qiita.com/shimopino/items/5fee7504c7acf044a521)
* [機械学習で特徴量を正しく選択する方法](https://rightcode.co.jp/blog/information-technology/feature-selection-right-choice)
* [特徴選択とは？機械学習の予測精度を改善させる必殺技「特徴選択」を理解しよう](https://www.codexa.net/feature-selection-methods/)
* [Human Activity Recognition Using Smartphones Data Set](https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones)
* [Human Activity Recognition using Smartphone](https://arxiv.org/abs/1401.8212)
* [Human Activity Analysis and Recognition from
Smartphones using Machine Learning Techniques](https://arxiv.org/abs/2103.16490)
* [Human Activity Recognition using Machine Learning](https://github.com/sushantdhumak/Human-Activity-Recognition-with-Smartphones)
* [How to Choose a Feature Selection Method For Machine Learning](https://machinelearningmastery.com/feature-selection-with-real-and-categorical-data/)
* [統計分析を理解しよう-よく使われている統計分析方法の概要-](https://www.nli-research.co.jp/report/detail/id=61928?site=nli)
* [Monash, UEA & UCR Time Series Extrinsic Regression Repository](http://tseregression.org/)
* [Flood Modeling Dataset 1](https://zenodo.org/record/3902694#.YTQjG50zaUk)
* [Flood Modeling Dataset 2](https://zenodo.org/record/3902696#.YTQktZ0zaUk)
* [Flood Modeling Dataset 3](https://zenodo.org/record/3902698#.YTQktZ0zaUk)
* [Monash University, UEA, UCR Time Series Extrinsic Regression Archive](https://arxiv.org/abs/2006.10996)
* [Time Series Extrinsic Regression](https://arxiv.org/abs/2006.12672)
* [ChangWeiTan/TS-Extrinsic-Regression](https://github.com/ChangWeiTan/TS-Extrinsic-Regression)